# Iterative Closest Point

*Aljoscha Rheinwalt, <aljoscha.rheinwalt@uni-potsdam.de> (2022)*

## Introduction

This lab is about the basics of the Singular Value Decomposition (SVD) based Iterative Closest Point (ICP) point cloud alignment approach. The goal is to develop our own implementation using only Numpy, which we can then illustrate and test on synthetic point clouds. Our toy model for this lab is a Gaussian hill which is a smooth and easy to sample surface, because the elevation $z$ of the hill can be expressed as a function of the coordinates $x$ and $y$:
$$z(x, y) = e^{-x^2-y^2} \;.$$

A synthetic point cloud $p$ from this model can be generated by randomly sampling $(x, y)$ coordinates and then calculating the corresponding $z$ coordinates from the model function. For testing purposes a second synthetic point cloud $q$ that is not aligned to the first can be retrieved from the first by rotating and translating it in three dimensions.

In [1]:
import numpy as np

# first point cloud p
n = 1000
x = np.random.random(n) * 2 - 1
y = np.random.random(n) * 2 - 1
z = np.exp(- x * x - y * y)
p = np.c_[x, y, z]

def rot3d(p, yaw, pitch, roll):
    a = np.array([[+np.cos(yaw), -np.sin(yaw), 0],
                  [+np.sin(yaw), +np.cos(yaw), 0],
                  [0, 0, 1]])
    b = np.array([[+np.cos(pitch), 0, +np.sin(pitch)],
                  [0, 1, 0],
                  [-np.sin(pitch), 0, +np.cos(pitch)]])
    c = np.array([[1, 0, 0],
                  [0, +np.cos(roll), -np.sin(roll)],
                  [0, +np.sin(roll), +np.cos(roll)]])
    return a.dot(b.dot(c.dot(p.T))).T


# second point cloud q
q = rot3d(p, np.pi/4, np.pi/4, np.pi/4)
q += np.array([0.25, 0.50, 0.75])

## Visualisation

There are probably more ways to visualize a 3D point cloud than there are ways to align them. All have their advantages and disadvantages. Although Matplotlib is probably the most common visualisation Python module, its 3D rendering capabilities are not very suitable for larger point clouds. Here we'll use Open3d for visualisation.

In [10]:
import open3d as o3d

# Open3d point cloud objects
po = o3d.geometry.PointCloud()
po.points = o3d.utility.Vector3dVector(p)
po.paint_uniform_color([1, 0.5, 0])

qo = o3d.geometry.PointCloud()
qo.points = o3d.utility.Vector3dVector(q)
qo.paint_uniform_color([0.5, 0, 1])

# open interactive visualization window
o3d.visualization.draw_geometries([po, qo])

## Absolute Orientation Problem and Alignment

Since our point clouds $p$ and $q$ are actually the same point cloud in two different coordinate systems, there is a rigid-body transformation that transforms one exactly into the other. Finding this transformation is called absolute orientation problem and for known associations, i.e., which point in q corresponds to which point in p, there are direct solutions. One direct approach is to translate by the difference between centroids and then decompose the cross co-variance matrix in its rotational parts, combine those and take that as the rotational difference.

In [11]:
def cross_cov(p, q, matches=None):
    n = p.shape[1]
    assert q.shape[1] == n, "dimension mismatch in cross_cov()"
    if matches == None:
        matches = [(i, i) for i in range(p.shape[0])]
    cov = np.zeros((n, n))
    for i, j in matches:
        pi = p[i]
        qj = q[j]
        cov += np.outer(pi, qj)
    return cov


# centroids
pm = p.mean(axis=0)
qm = q.mean(axis=0)

# cross co-variance
cc = cross_cov(p - pm, q - qm)

# rotational parts u and v
u, s, v = np.linalg.svd(cc)

# rotation matrix
r = u.dot(v)

# translation vector
t = pm - r.dot(qm)

# transformation
q1 = r.dot(q.T).T + t

Let us see how $q1$ compares to $q$ and $p$:

In [12]:
# point cloud object
q1o = o3d.geometry.PointCloud()
q1o.points = o3d.utility.Vector3dVector(q1)
q1o.paint_uniform_color([0, 1, 0.5])

# visualize all three point clouds
o3d.visualization.draw_geometries([po, qo, q1o])

# the same with a tiny z offset
q1o = o3d.geometry.PointCloud()
q1o.points = o3d.utility.Vector3dVector(q1 + np.array([0, 0, 0.01]))
q1o.paint_uniform_color([0, 1, 0.5])
o3d.visualization.draw_geometries([po, qo, q1o])

# mean square error
print(np.mean((p - q1)**2))

1.4951071195475887e-31


This works perfectly because data associations are perfect. However, often associations are not known and have to be estimated. This leads to the concept of ICP:

1. Associations are estimated by proximity.
2. The resulting transformation brings point clouds closer together.
3. Iterate over previous steps until point clouds converge.

For one iteration and with a KD-tree for efficient nearest neighbor queries this could be implemented in the following way.

In [13]:
from scipy.spatial import cKDTree as kdtree

# compute nearest neighbors
tr = kdtree(p - pm)
dd, ii = tr.query(q - qm, k=1, workers=-1)

# match points in p to points in q
matches = [(ii[i], i) for i in range(ii.shape[0])]

# compute transformation
cc = cross_cov(p - pm, q - qm, matches)
u, s, v = np.linalg.svd(cc)
r = u.dot(v)
t = pm - r.dot(qm)
q1 = r.dot(q.T).T + t

# mean square error
print(np.mean((p - q1)**2))

0.0732030625502793


In [14]:
# point cloud object
q1o = o3d.geometry.PointCloud()
q1o.points = o3d.utility.Vector3dVector(q1)
q1o.paint_uniform_color([0, 1, 0.5])

# visualize all three point clouds
o3d.visualization.draw_geometries([po, qo, q1o])

## ICP Implementation

With a for loop over several iterations we can implement the ICP concept in the following function.

In [18]:
def icp_svd(p, q, iterations=10):
    def matches(tr, q):
        dd, ii = tr.query(q, k=1, workers=-1)
        return [(ii[i], i) for i in range(ii.shape[0])]

    nq = q.shape[0]
    qs = np.zeros((iterations * nq, 4))
    pm = p.mean(axis=0)
    tr = kdtree(p - pm)
    qj = q.copy()
    for j in range(iterations):
        qm = qj.mean(axis=0)
        cc = cross_cov(p - pm, qj - qm, matches(tr, qj - qm))
        u, s, v = np.linalg.svd(cc)
        r = u.dot(v)
        t = pm - r.dot(qm)
        qj = r.dot(qj.T).T + t
        qs[j*nq:(j+1)*nq] = np.c_[qj, j * np.ones(nq)]

    return qs


qs = icp_svd(p, q, iterations = 10)

That function returns not just the final transformed $q$, but also all intermediate versions. This way we can visualize the progress of the algorithm and how it converges. For the ten different $q$-transformations we color points according to its iteration in a color gradient.

In [19]:
from matplotlib import pyplot as pl

# color gradient
cmap = pl.cm.viridis

# point cloud object for icp svd iterations
qso = o3d.geometry.PointCloud()
qso.points = o3d.utility.Vector3dVector(qs[:,:3])
qso.colors = o3d.utility.Vector3dVector(cmap(qs[:,3]/qs[:,3].max())[:,:3])
o3d.visualization.draw_geometries([po, qo, qso])

We can also color each point of all $q$-transformations according to its distance $d$ to $p$.

In [ ]:
# number of iterations (e.g., 10)
niter = qs.shape[0]//p.shape[0]

# distance d
d = np.sqrt(np.sum((qs[:,:3] - np.tile(p, (niter, 1)))**2, axis = 1))

# color gradient
cmap = pl.cm.magma_r

# point cloud object for icp svd iterations
qso = o3d.geometry.PointCloud()
qso.points = o3d.utility.Vector3dVector(qs[:,:3])
qso.colors = o3d.utility.Vector3dVector(cmap(d/d.max())[:,:3])
o3d.visualization.draw_geometries([po, qo, qso])

# mean distance per iteration
d.shape = (niter, p.shape[0])
d = np.mean(d, axis = 1)

fg, ax = pl.subplots()
ax.plot(d)
ax.set_xlabel("Iterations")
ax.set_ylabel("Mean distance")
ax.set_title("Convergence")